In [3]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-25 15:26:00.697917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 15:26:00.698056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 15:26:00.701877: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 15:26:00.729366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 15:26:04.447471: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../HAM10000/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/10015
[INFO]: Processed 2000/10015
[INFO]: Processed 3000/10015
[INFO]: Processed 4000/10015
[INFO]: Processed 5000/10015
[INFO]: Processed 6000/10015
[INFO]: Processed 7000/10015
[INFO]: Processed 8000/10015
[INFO]: Processed 9000/10015
[INFO]: Processed 10000/10015
(10015, 224, 224, 3)
(10015,)


In [8]:
from tensorflow.keras.applications import  ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
with tf.device('/CPU:0'):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    model = Model(inputs=base_model.input, outputs=base_model.output)
    features = model.predict(data, batch_size=32, verbose=1)

# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-21 22:07:35.100154: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1288 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0


313/313 [==============================] - 552s 2s/step
(10015, 100352)


In [9]:
data = features_flatten

In [10]:
num_classes = len(np.unique(labels))

In [11]:
import gc
import os

def calculate_sensitivity_specificity(y_true, y_pred, class_label):
    # Create a binary confusion matrix for the specified class
    true_positive = sum((y_true == class_label) & (y_pred == class_label))
    false_positive = sum((y_true != class_label) & (y_pred == class_label))
    true_negative = sum((y_true != class_label) & (y_pred != class_label))
    false_negative = sum((y_true == class_label) & (y_pred != class_label))

    # Calculate sensitivity and specificity
    sensitivity = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0

    return sensitivity, specificity
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":
        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = to_categorical(labels[train_index]), to_categorical(labels[test_index])

                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                #train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                #train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])


                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                # print(y_test)
                # print(y_test_encoded)
                # print(y_pred)
                # print(predictions)
                # Calculate evaluation metrics for the current fold
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivities = []
                specificities = []
                for class_label in range(num_classes):
                    sen, spe = calculate_sensitivity_specificity(y_test_encoded, y_pred, class_label)
                    sensitivities.append(sen)
                    specificities.append(spe)
                sensitivity = sum(sensitivities) / num_classes
                specificity = sum(specificities) / num_classes
                auc_roc = roc_auc_score(y_test, predictions,multi_class='ovr')
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, average='macro')
                f1 = f1_score(y_test_encoded, y_pred, average='macro')
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_prob = model.predict_proba(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivities = []
            specificities = []
            for class_label in range(num_classes):
                sen, spe = calculate_sensitivity_specificity(y_test, y_pred, class_label)
                sensitivities.append(sen)
                specificities.append(spe)
            sensitivity = sum(sensitivities) / num_classes
            specificity = sum(specificities) / num_classes
            auc_roc = roc_auc_score(y_test, y_prob,multi_class='ovr')
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [12]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8243512974051896
Sensitivity: 0.651794726528202
Specificity: 0.9545342789066538
AUC-ROC: 0.9537262342921559
MCC: 0.6596171513200884
Precision: 0.7489202562064737
F1 Score: 0.6883077822682954

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8153692614770459
Sensitivity: 0.6049248778459866
Specificity: 0.9480504189215847
AUC-ROC: 0.9545313047240713
MCC: 0.6310714006299426
Precision: 0.721813570618605
F1 Score: 0.6515649796932391

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.8171828171828172
Sensitivity: 0.6725668982298683
Specificity: 0.9475836730918061
AUC-ROC: 0.9608789142144387
MCC: 0.6357023069481247
Precision: 0.7027372333369686
F1 Score: 0.6828836704345189

[INFO] Fold 9 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8171828171828172
Sensitivity: 0.564086198482293
Specificity: 0.9495584357064036
AUC-ROC: 0.9462221202711918
MCC: 0.6355996586787875
Precision: 0.6028013927692901
F1 Score: 0.5794800211418834

[INFO] Fold 10 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8101898101898102
Sensitivity: 0.6151383079021672
Specificity: 0.9518105855938016
AUC-ROC: 0.9507812703480225
MCC: 0.6301696794834978
Precision: 0.675033263769793
F1 Score: 0.6398682421400999


In [13]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6347305389221557
Sensitivity: 0.24740135486404144
Specificity: 0.9015427568243265
AUC-ROC: 0.5753360172102627
MCC: 0.2718334811472551
Precision: 0.2510245965936278
F1 Score: 0.24583101963433476

[INFO] Fold 2 / 10 for NB


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.627744510978044
Sensitivity: 0.23852554598823256
Specificity: 0.8979439114954756
AUC-ROC: 0.5704380481220285
MCC: 0.24985418043663385
Precision: 0.24582840766901423
F1 Score: 0.24025263016701143

[INFO] Fold 3 / 10 for NB


...

Accuracy: 0.6223776223776224
Sensitivity: 0.22565679431638927
Specificity: 0.90229449173305
AUC-ROC: 0.5661058618378898
MCC: 0.2554112665326149
Precision: 0.22068425209600329
F1 Score: 0.2228165309868572

[INFO] Fold 8 / 10 for NB


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6103896103896104
Sensitivity: 0.24638901478342173
Specificity: 0.8960411777079823
AUC-ROC: 0.5734453121926377
MCC: 0.2352636770172518
Precision: 0.23882113113639675
F1 Score: 0.2424014708701287

[INFO] Fold 9 / 10 for NB


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.6483516483516484
Sensitivity: 0.25318262601290564
Specificity: 0.9072656564691676
AUC-ROC: 0.5829724187282302
MCC: 0.29740172789069225
Precision: 0.2506706278469975
F1 Score: 0.25135136434341243

[INFO] Fold 10 / 10 for NB
Accuracy: 0.6243756243756243
Sensitivity: 0.2536775128193039
Specificity: 0.9004660413521025
AUC-ROC: 0.5785171883277588
MCC: 0.2575010184590156
Precision: 0.2631081204595764
F1 Score: 0.2559302730908138


In [14]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.7245508982035929
Sensitivity: 0.4251637670614216
Specificity: 0.9255679422653043
AUC-ROC: 0.8076452540305592
MCC: 0.4489969811161218
Precision: 0.525586920610128
F1 Score: 0.43594559423992696

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.7425149700598802
Sensitivity: 0.4516221192127376
Specificity: 0.9259918472639793
AUC-ROC: 0.8383532818633065
MCC: 0.4762932603329219
Precision: 0.6305386840178377
F1 Score: 0.4728939895818372

[INFO] Fold 3 / 10 for KNN


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7345309381237525
Sensitivity: 0.355435004688736
Specificity: 0.9265191668527272
AUC-ROC: 0.7983632818454426
MCC: 0.4594246762503125
Precision: 0.37715421715932
F1 Score: 0.35272196529597216

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.7215568862275449
Sensitivity: 0.3662878584269529
Specificity: 0.9225768836340836
AUC-ROC: 0.8202048843850198
MCC: 0.43327848135063385
Precision: 0.4499135260360022
F1 Score: 0.37353544972796016

[INFO] Fold 5 / 10 for KNN


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7514970059880239
Sensitivity: 0.4051568540978239
Specificity: 0.933195104818734
AUC-ROC: 0.8303647471868357
MCC: 0.4989761448147477
Precision: 0.5105666175131175
F1 Score: 0.4079307654090973

[INFO] Fold 6 / 10 for KNN
Accuracy: 0.7422577422577422
Sensitivity: 0.4411811224989045
Specificity: 0.9248642056598454
AUC-ROC: 0.8192592034059308
MCC: 0.4696462162057538
Precision: 0.5563777285796772
F1 Score: 0.4552739297334763

[INFO] Fold 7 / 10 for KNN
Accuracy: 0.7332667332667333
Sensitivity: 0.395496095082815
Specificity: 0.9211383205264358
AUC-ROC: 0.8394941779321863
MCC: 0.4476635017474423
Precision: 0.5935090625805495
F1 Score: 0.402162960994836

[INFO] Fold 8 / 10 for KNN
Accuracy: 0.7262737262737263
Sensitivity: 0.411333957597217
Specificity: 0.9225218048559615
AUC-ROC: 0.8071611737077589
MCC: 0.4409813764963909
Precision: 0.5640802839193774
F1 Score: 0.42870995678448404

[INFO] Fold 9 / 10 for KNN
Accuracy: 0.7422577422577422
Sensitivity: 0.42250961950817306
Specificity: 

In [15]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7095808383233533
Sensitivity: 0.22439856096572516
Specificity: 0.8886042472560017
AUC-ROC: 0.8707229341474214
MCC: 0.3133956728141519
Precision: 0.40036936851902516
F1 Score: 0.2445006590897001

[INFO] Fold 2 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.719560878243513
Sensitivity: 0.237834271416361
Specificity: 0.8907242772556581
AUC-ROC: 0.8877936075612111
MCC: 0.34498769296967335
Precision: 0.49808829962140067
F1 Score: 0.2658561439487163

[INFO] Fold 3 / 10 for Random Forest


...

Accuracy: 0.7072927072927073
Sensitivity: 0.219212900047037
Specificity: 0.8860147451151731
AUC-ROC: 0.8949151534095507
MCC: 0.2987015247408423
Precision: 0.4355058192510593
F1 Score: 0.2396465730926011

[INFO] Fold 9 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7092907092907093
Sensitivity: 0.21491220012796677
Specificity: 0.8860302041039742
AUC-ROC: 0.8640657375146049
MCC: 0.3033735507268436
Precision: 0.5051749000128456
F1 Score: 0.23440654345064946

[INFO] Fold 10 / 10 for Random Forest


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7122877122877123
Sensitivity: 0.22666081649290604
Specificity: 0.8876637929261489
AUC-ROC: 0.8725874279898015
MCC: 0.31750195984964885
Precision: 0.4728740719128645
F1 Score: 0.2528587006612664


In [ ]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1, force_col_wise= True)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Total Bins 16605785
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 99318
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.970858
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.471694
[LightGBM] [Info] Start training from score -2.196670
[LightGBM] [Info] Start training from score -0.401092
[LightGBM] [Info] Start training from score -4.254393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.7954091816367266
Sensitivity: 0.45346113768288604
Specificity: 0.9357459693141991
AUC-ROC: 0.9401777284158493
MCC: 0.5763068652513839
Precision: 0.7753341494007173
F1 Score: 0.517165654771803

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Total Bins 16590816
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 99331
[LightGBM] [Info] S

/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7674650698602794
Sensitivity: 0.36270986644120967
Specificity: 0.9244238937934895
AUC-ROC: 0.9221463203851396
MCC: 0.503842778355078
Precision: 0.48778139899812906
F1 Score: 0.4029652870302351

[INFO] Fold 4 / 10 for LGBM
[LightGBM] [Info] Total Bins 16596244
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 99319
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.968696
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.471694
[LightGBM] [Info] Start training from score -2.197668
[LightGBM] [Info] Start training from score -0.401092
[LightGBM] [Info] Start training from score -4.254393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7924151696606786
Sensitivity: 0.42851503559361326
Specificity: 0.931145551464496
AUC-ROC: 0.9419628891329532
MCC: 0.5629262308934534
Precision: 0.6495717200150697
F1 Score: 0.4918486777932363

[INFO] Fold 5 / 10 for LGBM
[LightGBM] [Info] Total Bins 16614269
[LightGBM] [Info] Number of data points in the train set: 9013, number of used features: 99348
[LightGBM] [Info] Start training from score -3.422843
[LightGBM] [Info] Start training from score -2.968696
[LightGBM] [Info] Start training from score -2.209729
[LightGBM] [Info] Start training from score -4.462032
[LightGBM] [Info] Start training from score -2.197668
[LightGBM] [Info] Start training from score -0.401258
[LightGBM] [Info] Start training from score -4.254393


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7964071856287425
Sensitivity: 0.42205975534774265
Specificity: 0.9332566038097976
AUC-ROC: 0.9401168164724226
MCC: 0.571295662795938
Precision: 0.6495886572932609
F1 Score: 0.48307135835256404

[INFO] Fold 6 / 10 for LGBM
[LightGBM] [Info] Total Bins 16605900
[LightGBM] [Info] Number of data points in the train set: 9014, number of used features: 99335
[LightGBM] [Info] Start training from score -3.422954
[LightGBM] [Info] Start training from score -2.968807
[LightGBM] [Info] Start training from score -2.208829
[LightGBM] [Info] Start training from score -4.462143
[LightGBM] [Info] Start training from score -2.197779
[LightGBM] [Info] Start training from score -0.401369
[LightGBM] [Info] Start training from score -4.254504
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/opt/tljh/user/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.7892107892107892
Sensitivity: 0.41742123331783937
Specificity: 0.9317545767596073
AUC-ROC: 0.9336000952282019
MCC: 0.5553622209480056
Precision: 0.5968215664644235
F1 Score: 0.4714271907032538

[INFO] Fold 7 / 10 for LGBM
[LightGBM] [Info] Total Bins 16636941
[LightGBM] [Info] Number of data points in the train set: 9014, number of used features: 99381
[LightGBM] [Info] Start training from score -3.422954
[LightGBM] [Info] Start training from score -2.968807
[LightGBM] [Info] Start training from score -2.209840
[LightGBM] [Info] Start training from score -4.462143
[LightGBM] [Info] Start training from score -2.196781
[LightGBM] [Info] Start training from score -0.401369
[LightGBM] [Info] Start training from score -4.254504


In [ ]:
# Save results to a CSV file
results_df.to_csv('DS7_ResNet_LGBM.csv', index=False)

In [ ]:
results_df